### Atividade 5 - Tópicos em Controle e Automação Industrial (mestrado e doutorado em engenharia elétrica - PPGEE/UFAM)
### Professor: Kenny Vinente dos Santos
#### Alunas: 
#### Cristiane Brasil Ueda,      Matrícula: 2240543
#### Thaiane Naiara Siqueira de Jesus,   Matrícula: 2240930

In [11]:
from pulp import LpProblem, LpVariable, LpMinimize, value, PULP_CBC_CMD
import numpy as np

# Criar o modelo de otimização
model = LpProblem("Problema_Hidrotermico", LpMinimize)

# Variáveis de decisão
pt11 = LpVariable("pt11", lowBound=0)   # Geração térmica 1 no mês 1
pt21 = LpVariable("pt21", lowBound=0)   # Geração térmica 2 no mês 1
pt12 = LpVariable("pt12", lowBound=0)   # Geração térmica 1 no mês 2
pt22 = LpVariable("pt22", lowBound=0)   # Geração térmica 2 no mês 2
pt32 = LpVariable("pt32", lowBound=0)   # Geração térmica 3 no mês 2
deficit1 = LpVariable("deficit1", lowBound=0)  # Déficit de energia no mês 1
deficit2 = LpVariable("deficit2", lowBound=0)  # Déficit de energia no mês 2
q1 = LpVariable("q1", lowBound=0)       # Vazão turbinada no mês 1
q2 = LpVariable("q2", lowBound=0)       # Vazão turbinada no mês 2
v1 = LpVariable("v1", lowBound=0)       # Volume final no mês 1
v2 = LpVariable("v2", lowBound=0)       # Volume final no mês 2

# Função objetivo: minimizar o custo total
model += (
    10 * pt11 + 12 * pt21 + 10 * pt12 + 12 * pt22 + 20 * pt32 +
    500 * deficit1 + 500 * deficit2
), "Custo_Total"

# Restrições de atendimento à carga
model += pt11 + pt21 + 0.96 * q1 + deficit1 == 1000, "Carga_Mes1"
model += pt12 + pt22 + pt32 + 0.96 * q2 + deficit2 == 1000, "Carga_Mes2"

# Restrições de balanço hídrico
model += v1 == 2050 + 150 - q1, "Balanco_Hidrico_Mes1"
model += v2 == v1 + 450 - q2, "Balanco_Hidrico_Mes2"

# Limites de volume
model += v1 <= 4100, "Volume_Max_Mes1"
model += v2 <= 4100, "Volume_Max_Mes2"

# Resolver o problema
model.solve(PULP_CBC_CMD(msg=False))

# Exibir os resultados
print("Status da Otimização:", model.status)
print("Custo Total: R$", value(model.objective))
print("pt11 (Mês 1, Térmica 1):", pt11.varValue, "MWmês")
print("pt21 (Mês 1, Térmica 2):", pt21.varValue, "MWmês")
print("pt12 (Mês 2, Térmica 1):", pt12.varValue, "MWmês")
print("pt22 (Mês 2, Térmica 2):", pt22.varValue, "MWmês")
print("pt32 (Mês 2, Térmica 3):", pt32.varValue, "MWmês")
print("Déficit Mês 1:", deficit1.varValue, "MWmês")
print("Déficit Mês 2:", deficit2.varValue, "MWmês")
print("q1 (Vazão turbinada, Mês 1):", q1.varValue, "m³/s")
print("q2 (Vazão turbinada, Mês 2):", q2.varValue, "m³/s")
print("v1 (Volume final, Mês 1):", v1.varValue, "hm³")
print("v2 (Volume final, Mês 2):", v2.varValue, "hm³")

# Construir a matriz tecnológica
A = [
    [1, 1, 0, 0, 0, 1, 0, 0.96, 0, 0, 0],  # Restrição de carga mês 1
    [0, 0, 1, 1, 1, 0, 1, 0, 0.96, 0, 0],  # Restrição de carga mês 2
    [0, 0, 0, 0, 0, 0, 0, -1, 0, 1, 0],    # Balanço hídrico mês 1
    [0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 1],   # Balanço hídrico mês 2
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],     # Limite de volume mês 1
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]      # Limite de volume mês 2
]

# Exibir a matriz tecnológica
A = np.array(A)
print("\nMatriz Tecnológica (A):")
print(A)


Status da Otimização: 1
Custo Total: R$ 0.0
pt11 (Mês 1, Térmica 1): 0.0 MWmês
pt21 (Mês 1, Térmica 2): 0.0 MWmês
pt12 (Mês 2, Térmica 1): 0.0 MWmês
pt22 (Mês 2, Térmica 2): 0.0 MWmês
pt32 (Mês 2, Térmica 3): 0.0 MWmês
Déficit Mês 1: 0.0 MWmês
Déficit Mês 2: 0.0 MWmês
q1 (Vazão turbinada, Mês 1): 1041.6667 m³/s
q2 (Vazão turbinada, Mês 2): 1041.6667 m³/s
v1 (Volume final, Mês 1): 1158.3333 hm³
v2 (Volume final, Mês 2): 566.66667 hm³

Matriz Tecnológica (A):
[[ 1.    1.    0.    0.    0.    1.    0.    0.96  0.    0.    0.  ]
 [ 0.    0.    1.    1.    1.    0.    1.    0.    0.96  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.   -1.    0.    1.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.   -1.   -1.    1.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.  ]]


Os valores das variáveis pt1_1, pt2_1, pt3_1, pt1_2, pt2_2 e pt3_2 sendo todos iguais a zero no resultado indicam que o modelo determinou que a geração térmica não era necessária para atender à demanda energética em ambos os meses. Isso ocorre porque a hidrelétrica conseguiu suprir toda a demanda ao menor custo possível, o Custo_Total deu 0.0, isso indica que, na solução do modelo, nenhuma das térmicas foi acionada, e o modelo não incluiu custos associados à geração hidrelétrica.

## Problema de Otimização com Dois Estágios e Incertezas
    


In [15]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, value
import numpy as np

# Dados do problema
c_t = [10, 20, 40, 80]  # Custo incremental das usinas térmicas (R$/MWmês)
max_gen_t = [100, 150, 200, 250]  # Limite de geração térmica (MWmês)
max_q = 1500  # Vazão máxima turbinada (m³/s)
max_v = 4100  # Volume máximo do reservatório (hm³)
prod = 0.96  # Produtividade (MWmês por m³/s)
max_gen_h = 1440  # Geração máxima hidrelétrica (MWmês)
load = 1000  # Demanda fixa (MWmês)
rho = 0.96  # Conversão de vazão para energia

# Estados dos fluxos hidrológicos (dois cenários equiprováveis)
y1 = 150  # Primeiro estágio
y2_scenarios = [450, 300]  # Segundo estágio: dois cenários

# Probabilidades dos cenários
prob_scenarios = [0.5, 0.5]

# Definir o problema de otimização
prob = LpProblem("Problema_2_estagios_incerteza", LpMinimize)

# Variáveis de decisão - Primeiro estágio
p_t1 = [LpVariable(f"p_t1_{i+1}", 0, max_gen_t[i]) for i in range(4)]  # Geração térmica 1
q1 = LpVariable("q1", 0, max_q)  # Vazão turbinada 1
v1 = LpVariable("v1", 0, max_v)  # Volume do reservatório 1
d1 = LpVariable("d1", 0)  # Déficit de energia 1

# Variáveis de decisão - Segundo estágio (cenários)
p_t2 = [[LpVariable(f"p_t2_{i+1}_s{j+1}", 0, max_gen_t[i]) for i in range(4)] for j in range(2)]
q2 = [LpVariable(f"q2_s{j+1}", 0, max_q) for j in range(2)]
v2 = [LpVariable(f"v2_s{j+1}", 0, max_v) for j in range(2)]
d2 = [LpVariable(f"d2_s{j+1}", 0) for j in range(2)]

# Função objetivo
prob += (
    lpSum(c_t[i] * p_t1[i] for i in range(4)) + d1 * 500 +  # Custo do primeiro estágio
    lpSum(prob_scenarios[j] * (lpSum(c_t[i] * p_t2[j][i] for i in range(4)) + d2[j] * 500) for j in range(2))
)

# Restrições do primeiro estágio
prob += lpSum(p_t1) + rho * q1 + d1 == load, "Restricao_demanda_1"
prob += v1 + q1 * y1 == max_v, "Balanco_hidrologico_1"

# Restrições do segundo estágio (para cada cenário)
for j in range(2):
    prob += lpSum(p_t2[j]) + rho * q2[j] + d2[j] == load, f"Restricao_demanda_2_s{j+1}"
    prob += v2[j] + q2[j] * y2_scenarios[j] == v1, f"Balanco_hidrologico_2_s{j+1}"

# Resolver o problema
prob.solve()

# Extração da matriz tecnológica (A)
constraints = list(prob.constraints.values())
variables = prob.variables()
num_constraints = len(constraints)
num_vars = len(variables)

# Criar matriz tecnológica A
A = np.zeros((num_constraints, num_vars))

# Preencher a matriz tecnológica
for i, constraint in enumerate(constraints):
    for j, var in enumerate(variables):
        A[i, j] = constraint.get(var, 0)

# Resultado
print("Matriz Tecnológica (A):")
print(A)
print("\nValores Ótimos das Variáveis:")
for var in variables:
    print(f"{var.name}: {var.varValue}")
print("\nCusto Total Mínimo:", value(prob.objective))


Matriz Tecnológica (A):
[[  1.     0.     0.     1.     1.     1.     1.     0.     0.     0.
    0.     0.     0.     0.     0.     0.96   0.     0.     0.     0.
    0.  ]
 [  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.   150.     0.     0.     1.     0.
    0.  ]
 [  0.     1.     0.     0.     0.     0.     0.     1.     0.     1.
    0.     1.     0.     1.     0.     0.     0.96   0.     0.     0.
    0.  ]
 [  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.   450.     0.    -1.     1.
    0.  ]
 [  0.     0.     1.     0.     0.     0.     0.     0.     1.     0.
    1.     0.     1.     0.     1.     0.     0.     0.96   0.     0.
    0.  ]
 [  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.   300.    -1.     0.
    1.  ]]

Valores Ótimos das Variáveis:
d1: 273.76
d2_s1: 300.0
d2_s2: 300.0
p_t1_1: